<a href="https://colab.research.google.com/github/ryganon/projetos_IAI/blob/master/Sistemas_Multi_Agentes/Simula%C3%A7%C3%A3o_Multiagente_MESA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install mesa

In [ ]:
import matplotlib as plt
import pandas as pd
import numpy as np

from mesa import Agent, Model
from mesa.time import RandomActivation

# Organizando dados

In [ ]:
dados = pd.read_csv("acoes_indice_bovespa_indicadores.csv")

In [ ]:
dados["DATAPREGAO"] = pd.to_datetime(dados['DATAPREGAO'])

# Simulação de Agentes Financeiros



## Classe BrokerAgent

Classe responsável por coletar os dados da BOVESPA e disponibilizar para os demais agentes da simulação.

**Funções:**

> get_stocks()

> get_stocks_by_date








In [ ]:
class BrokerAgent:
    """ Agente do tipo Broker que retem as informações dos dados
        da Bovespa
    """
    def __init__(self, data):
        self.stocks = data
        self.global_stocks = {item: 10000 for item in list(self.stocks["CODNEG"].unique())}

    def get_stocks(self):
        return self.stocks

    def get_stocks_by_date(self, ref_date, selected_stocks):

        temp = self.stocks.loc[self.stocks["DATAPREGAO"] <= ref_date]
        return temp.loc[temp["CODNEG"].isin(selected_stocks)]

    def get_global_stocks(self):
        return self.global_stocks  

## Classe GenericTraderAgent

Classe para implementação de todas as funções comuns a todos os agentes. 

A classe serve como template para a criação dos agentes especializados.

**Variáveis de classe:**

> broker

> stocks

> trading_amount (10000)

> portfolio_profit

**Funções:**

> **get_stocks():** consulta um objeto **BrokerAgent** para receber os dados da Bovespa.

> **buy_stock():** efetiva a compra de uma quantidade de ações considerando dinheiro disponível e o valor da ação no dia.

> **sell_stock():** vende uma quantidade de ações considerando o valor da ação no dia.

In [ ]:
class GenericTraderAgent(Agent):
    """ Agente de trading responsável por realizar compra e venda 
        Template para construção de novos agentes.
        Objetos que herdam GenericTraderAgent devem implementar:
        - função step() com estratégia
    """

    def __init__(self, unique_id, model, broker, list_stocks):
        super().__init__(unique_id, model)

        # agente broker
        self.broker = broker
        
        # inicializa portfólio
        self.stocks = {item: 0 for item in list(self.broker.get_stocks()["CODNEG"].unique())}
        
        # valor de investimento inicial
        self.trading_amount = 100000
        
        # valor total da carteira
        self.portfolio_profit = {item: 0 for item in list(self.broker.get_stocks()["CODNEG"].unique())}

        # ações para monitorar
        self.list_stocks = list_stocks


    def get_stocks(self):
        # a data de referência para as ações
        ref_date = self.model.actual_date
        selected_stocks = self.list_stocks #["MGLU3", "VVAR3"] # teste

        return self.broker.get_stocks_by_date(ref_date, selected_stocks)

    def buy_stock(self, buy_data, qty):
        final_price = buy_data["PREULT"] * qty

        # se houver dinheiro
        if final_price < self.trading_amount:

            # adiciona qtde no portifolio
            self.stocks[buy_data["CODNEG"]] += qty 
            # desconta o valor da compra do amount
            self.trading_amount -= final_price 

            # atualiza a carteira
            # valor = total de acoes no portfolio * valor de ultima negociacao
            self.portfolio_profit[buy_data["CODNEG"]] = self.stocks[buy_data["CODNEG"]] * buy_data["PREULT"]
            
            print('{} \t Agente: {} \t < COMPRA {} > \t {} \t PREÇO AÇÃO = {:.2f}'.format(buy_data["DATAPREGAO"], 
                                                                                          self.unique_id, 
                                                                                          qty, 
                                                                                          buy_data["CODNEG"], 
                                                                                          buy_data["PREULT"]))
        
    def sell_stock(self, sell_data, qty):
        final_price = sell_data["PREULT"] * qty

        # remove qtde no portifolio
        self.stocks[sell_data["CODNEG"]] -= qty 
        
        # retorna o valor da venda do amount
        self.trading_amount += final_price 

        # atualiza a carteira
        # valor = total de acoes no portfolio * valor de ultima negociacao
        self.portfolio_profit[sell_data["CODNEG"]] = self.stocks[sell_data["CODNEG"]] * sell_data["PREULT"]
            
        print('{} \t Agente: {} \t < VENDA {} > \t {} \t PREÇO AÇÃO = {:.2f}'.format(sell_data["DATAPREGAO"], 
                                                                                     self.unique_id, 
                                                                                     qty, 
                                                                                     sell_data["CODNEG"], 
                                                                                     sell_data["PREULT"]))

## Classe SimpleTraderAgent

Classe especializada na estratégia de cruzamento de médias móveis.

Herda a classe **GenericTraderAgent**.

**Variáveis de classe:**

> Herdadas de **GenericTraderAgent**.

**Funções:**

> **step():** Método do framework MESA para indicar o avanço de uma iteração na simulação. Local onde serão chamadas as funções de estratégia.

> **main_strategy():** Implementação da estratéégia de compra e venda do Agente.

In [ ]:
class SimpleTraderAgent(GenericTraderAgent):
    """ Agente de negociação simples.
        Estratégia: cruzamento de médias móveis.
    """

    def __init__(self, unique_id, model, broker, list_stocks):
        super().__init__(unique_id, model, broker, list_stocks)

    def step(self):
        #print(str(self.unique_id), "iniciei", self.model.actual_date)
        self.main_strategy()

    # cruzamento de medias moveis
    def main_strategy(self):

        # pegando os dados do papel
        hist_data = self.get_stocks()
        
        
        # verificando o valor atual da ação 
        temp_data = hist_data.loc[hist_data["DATAPREGAO"] == self.model.actual_date]
  
        for index, row in temp_data.iterrows():
            if row["SMA7"] > row["SMA14"]:
                # executa compra
                self.buy_stock(row, 100)
            else:
                self.sell_stock(row, 50)

## Classe MarketSimulationModel

Classe com o modelo de simulação multiagente utilizando o MESA.

Herda a classe **Model** (MESA).

**Variáveis de classe:**

> schedule

> trading_dates

> broker

> N agentes da simulação

**Funções:**

> **step():** Método do framework MESA para indicar o avanço de uma iteração na simulação. A execução invoca todos os agentes do ambiente.


In [ ]:
class MarketSimulationModel(Model):
    """ Modelo de simulação de Mercado Financeiro """
    
    def __init__(self, stock_data, trading_dates):

        # ativação dos agentes será aleatório 
        self.schedule = RandomActivation(self)
        
        # datas de negociação da simulação
        self.trading_dates = iter(trading_dates)

        # criando Broker
        broker = BrokerAgent(stock_data)

        # criando agentes
        list_stocks = ["MGLU3", "VVAR3"] # varejo
        ag = SimpleTraderAgent(0, self, broker, list_stocks)
        self.schedule.add(ag)

        list_stocks = ["ITSA4", "BBDC4"] # bancos 
        ag = SimpleTraderAgent(1, self, broker, list_stocks)
        self.schedule.add(ag)

        #ag = SimpleTraderAgent(2, self, broker)
        #self.schedule.add(ag)

    def step(self):
        '''Executa operações para um novo dia '''
        self.actual_date = next(self.trading_dates)
        self.schedule.step()

## Execução do sistema

Etapas:

* Definição das datas da simulação.
* Inicialização do modelo de simuação.
* Execução do processo iterativo de simulação por data.

In [ ]:
# Data inicial da simulação
start_at = pd.to_datetime("2019-10-20")
# Data final da simulação
end_at = pd.to_datetime("2019-11-25")

datas = list(pd.date_range(start_at, end_at))

In [ ]:
# criando modelo de simulação
modelo = MarketSimulationModel(dados, datas)

# total de agentes na simulação
total_agentes = len(modelo.schedule.agents)

# exibindo dados
for i in range(total_agentes):
    print("Agente ", i)
    print("\t Patrimônio inicial:", modelo.schedule.agents[i].trading_amount)
    print("\n")

# executando a simulação
for i in range(len(datas)):
    modelo.step()

# exibindo resultados
for i in range(total_agentes):
    print("\nAgente ", i, modelo.schedule.agents[i].list_stocks)
    amount = modelo.schedule.agents[i].trading_amount
    profit = sum(modelo.schedule.agents[i].portfolio_profit.values())
    print("\t Patrimônio final:", amount)
    print("\t Valor da carteira de ações:", profit)
    print("\t Total: ", amount + profit)


Agente  0
	 Patrimônio inicial: 100000


Agente  1
	 Patrimônio inicial: 100000


2019-10-21 00:00:00 	 Agente: 1 	 < COMPRA 100 > 	 BBDC4 	 PREÇO AÇÃO = 34.03
2019-10-21 00:00:00 	 Agente: 1 	 < COMPRA 100 > 	 ITSA4 	 PREÇO AÇÃO = 13.26
2019-10-21 00:00:00 	 Agente: 0 	 < COMPRA 100 > 	 MGLU3 	 PREÇO AÇÃO = 43.10
2019-10-21 00:00:00 	 Agente: 0 	 < COMPRA 100 > 	 VVAR3 	 PREÇO AÇÃO = 7.81
2019-10-22 00:00:00 	 Agente: 1 	 < COMPRA 100 > 	 BBDC4 	 PREÇO AÇÃO = 35.00
2019-10-22 00:00:00 	 Agente: 1 	 < COMPRA 100 > 	 ITSA4 	 PREÇO AÇÃO = 13.56
2019-10-22 00:00:00 	 Agente: 0 	 < COMPRA 100 > 	 MGLU3 	 PREÇO AÇÃO = 43.72
2019-10-22 00:00:00 	 Agente: 0 	 < COMPRA 100 > 	 VVAR3 	 PREÇO AÇÃO = 7.81
2019-10-23 00:00:00 	 Agente: 1 	 < COMPRA 100 > 	 BBDC4 	 PREÇO AÇÃO = 35.30
2019-10-23 00:00:00 	 Agente: 1 	 < COMPRA 100 > 	 ITSA4 	 PREÇO AÇÃO = 13.66
2019-10-23 00:00:00 	 Agente: 0 	 < COMPRA 100 > 	 MGLU3 	 PREÇO AÇÃO = 43.49
2019-10-23 00:00:00 	 Agente: 0 	 < COMPRA 100 > 	 VVAR3 	 PRE